# Work from ted talk video by data school
https://www.youtube.com/watch?v=dPwLlJkSHLo

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
pd.__version__

In [ ]:
%matplotlib inline

In [ ]:
ted = pd.read_csv("./ted.txt")


In [ ]:
ted.shape

## Data Check


In [ ]:
ted.isna().sum()

In [ ]:
# Check date - convert unix date to date time
ted['film_date'] = pd.to_datetime(ted.film_date, unit='s')
ted['published_date'] = pd.to_datetime(ted.published_date, unit='s')
ted['days_since_filmed'] = (ted['published_date'] - ted['film_date']).dt.days

ted['days_since_filmed'].describe()


In [ ]:
ted[ted['film_date'].dt.year >= 2017]
max(ted['film_date'])

# 2. Which talks provoke most online discussion?

In [ ]:
ted.sort_values('comments').tail()

In [ ]:
## Use number of views as we don't know how long talk was online
ted['comments_per_view'] = ted.comments / ted.views

In [ ]:
ted.sort_values('comments_per_view').tail()

In [ ]:
ted['views_per_comment'] = ted.views / ted.comments
ted.sort_values('views_per_comment').head()

### takeaways
Consider the limitations
    - no nested comments (detailed data required)
    - BIAS (normalise the data as per views per comments)

# 3. Visualise the distribtuion of comments

In [ ]:
ted.comments.plot(kind='hist')

In [ ]:
ted.loc[ted.comments < 1000, 'comments'].plot(kind='hist', bins = 20)

In [ ]:
# missing in above chart
ted[ted.comments >= 1000].shape

# 4. Plot the number of talks that took place each year

In [ ]:
ted.event

In [ ]:
ted.film_date.dt.year.value_counts().sort_index().plot(kind='line')

# 5. What were the best events in Ted history to attend?

In [ ]:
ted.event.value_counts().head()

In [ ]:
ted.groupby('event').views.mean().sort_values().tail()

In [ ]:
ted.groupby('event').views.agg(['count','mean']).sort_values('mean').tail()

In [ ]:
ted.groupby('event').views.agg(['count','mean','sum']).sort_values('sum').tail()

### think creatively the data you have to answer the question and include weaknesses
### check the sample size

## 6. Unpack the ratings data

In [ ]:
ted.ratings.head()

In [ ]:
print(type(ted.ratings[0]))
ted.ratings[0]

In [ ]:
import ast

In [ ]:
ast.literal_eval(ted.ratings[0])

In [ ]:
def str_to_list(ratings_str):
    return ast.literal_eval(ratings_str)

In [ ]:
str_to_list(ted.ratings[0])

In [ ]:
## alternative to above function
ted.ratings.apply(ast.literal_eval).head()

In [ ]:
ted['ratings_list'] = ted.ratings.apply(lambda x: ast.literal_eval(x))

In [ ]:
ted['ratings_list'][0]

In [ ]:
ted.dtypes


# 7. Count the total numbr of ratings received by each talk

- new column name 'num_ratings'


In [ ]:
ted.ratings_list[0]

In [ ]:
def get_num_ratings(list_of_dicts):
    num = 0
    for d in list_of_dicts:
        num += d['count']
    #return list_of_dicts[0]['count']
    return num

In [ ]:
print(get_num_ratings(ted.ratings_list[0]))
print('---------------')
# or
print(pd.DataFrame(ted.ratings_list[0])['count'].sum())

In [ ]:
ted['num_ratings'] = ted.ratings_list.apply(get_num_ratings)

In [ ]:
ted.num_ratings.describe()

# 8. Which Occupations deliver the funniest TED talks on average

- for each talk, calculate the most frequent rating
- for each talk, clean the occupation data so that theres only funny data

In [ ]:
ted.ratings.str.contains('Funny').value_counts()

In [ ]:
def get_funny_ratings(list_of_dicts):
    for d in list_of_dicts:
        if d['name'] == "Funny":
            return d['count']

In [ ]:
ted['funny_ratings'] = ted.ratings_list.apply(get_funny_ratings, "Funny")

In [ ]:
ted.funny_ratings.head()


In [ ]:
## How funny was the talk - Get the funny rating of all ratings per talk

In [ ]:
ted['funny_rate'] = ted.funny_ratings / ted.num_ratings

In [ ]:
ted.sort_values('funny_rate').speaker_occupation.tail(20)

In [ ]:
## Step 3 Analyse the funny rate by occupation 

In [ ]:
ted.groupby('speaker_occupation').funny_rate.mean().sort_values().tail()

In [ ]:
ted.groupby('speaker_occupation').funny_rate.agg(['count','mean']).sort_values('mean').tail()

In [ ]:
## step 4: Focus on occupation that are well represented in the data

In [ ]:
occupation_counts = ted.speaker_occupation.value_counts()

In [ ]:
top_occupations = occupation_counts[occupation_counts >= 5].index

In [ ]:
type(top_occupations)

In [ ]:
ted_top_occupations  = ted[ted.speaker_occupation.isin(top_occupations)]
ted_top_occupations.shape

In [ ]:
ted_top_occupations.groupby('speaker_occupation').funny_rate.mean().sort_values()